In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import (Dense, Conv2D , MaxPool2D , Flatten , Dropout ,
                          BatchNormalization,Input,AveragePooling2D,GlobalAveragePooling2D)
from tensorflow.keras.models import Model

from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
from keras.optimizers import SGD,Adam

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.efficientnet import EfficientNetB7
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [ ]:
main_path = '/kaggle/input/ranzcr-clip-catheter-line-classification'
out_path = '../input/ranzcr-clip-catheter-line-classification'

In [ ]:
os.listdir(out_path)

In [ ]:
for path in os.listdir(main_path):
    print(path)

In [ ]:
train_df = pd.read_csv(os.path.join(main_path,'train.csv'))

In [ ]:
train_df.head()

In [ ]:
train_annotate = pd.read_csv(os.path.join(main_path,'train_annotations.csv'))

In [ ]:
train_annotate.head()

In [ ]:
sample_df =  pd.read_csv(os.path.join(main_path,'sample_submission.csv'))

In [ ]:
sample_df.head()

In [ ]:
sample_df.shape

In [ ]:
def append_ext(fn):
    return fn+".jpg"

train_df["StudyInstanceUID"]=train_df["StudyInstanceUID"].apply(append_ext)

In [ ]:
# train_df["StudyInstanceUID"].tolist()

In [ ]:
datagen=ImageDataGenerator(rescale=1./255.,
                            validation_split=0.25,
                            featurewise_center=True,
                            featurewise_std_normalization=True,
                            rotation_range=20,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            horizontal_flip=True,
                            vertical_flip=True,
                            fill_mode="nearest"                            
                          )

train_generator=datagen.flow_from_dataframe(
dataframe=train_df,
directory="/kaggle/input/ranzcr-clip-catheter-line-classification/train/",
x_col="StudyInstanceUID",
y_col=['ETT - Abnormal', 'ETT - Borderline',
       'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal',
       'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present'],
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
color_mode='rgb',
target_size=(224,224))


val_generator=datagen.flow_from_dataframe(
dataframe=train_df,
directory="/kaggle/input/ranzcr-clip-catheter-line-classification/train/",
x_col="StudyInstanceUID",
y_col=['ETT - Abnormal', 'ETT - Borderline',
       'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal',
       'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present'],
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
color_mode='rgb',
target_size=(224,224))

In [ ]:
num_classes = 11
# model = Sequential()
# model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (150,150,1)))
# model.add(BatchNormalization())
# model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
# model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
# model.add(Dropout(0.1))
# model.add(BatchNormalization())
# model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
# model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
# model.add(BatchNormalization())
# model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
# model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())
# model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
# model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())
# model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
# model.add(Flatten())
# model.add(Dense(units = 128 , activation = 'relu'))
# model.add(Dropout(0.2))
# model.add(Dense(units = num_classes , activation = 'softmax'))
# model.compile(optimizer = "rmsprop" , loss = 'CategoricalCrossentropy' , metrics = ['accuracy'])
# model.summary()

In [ ]:
# def exponential_decay(lr0, s):
#     def exponential_decay_fn(epoch):
#         return lr0 * 0.1 **(epoch / s)
#     return exponential_decay_fn

# exponential_decay_fn = exponential_decay(0.01, 20)

# lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

learn_rate=.001
sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('model.h5', save_best_only=True, 
                                                   monitor='val_auc', mode='max')

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3,
                                                     restore_best_weights=True)
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_auc", patience=3, min_lr=1e-6, mode='max')

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name!='/device:GPU:0':
    raise SystemError('GPU Device not found')
print('Found GPU at:{}'.format(device_name))

In [ ]:
with tf.device('/gpu:0'):
    baseModel = EfficientNetB7(weights = None, include_top=False,
                  input_tensor=Input(shape=(224, 224, 3)),
                              drop_connect_rate=0.7)
    baseModel.load_weights('../input/tfkerasefficientnetimagenetnotop/efficientnetb7_notop.h5')
    headModel = baseModel.output
    headModel = GlobalAveragePooling2D()(headModel)
    headModel = Dense(1024, activation='relu')(headModel)
    headModel = Dense(num_classes, activation="sigmoid")(headModel)
    model = Model(inputs=baseModel.input, outputs=headModel)

    for layer in baseModel.layers:
        layer.trainable = False
    model.compile(loss="binary_crossentropy", optimizer='sgd',metrics=["accuracy"])
    history = model.fit(
            train_generator,
            epochs=20,
            batch_size = 32,
            validation_data=val_generator,
            callbacks=[checkpoint_cb,lr_reducer,early_stopping_cb])

In [ ]:
hist_df = pd.DataFrame(history.history)

In [ ]:
hist_df.head()

In [ ]:
# Training Vs Validation Accuracy
plt.plot(hist_df.accuracy)
plt.plot(hist_df.val_accuracy)

In [ ]:
#Training vs Validation Loss
plt.plot(hist_df.loss)
plt.plot(hist_df.val_loss)

In [ ]:
df_submission = sample_df.copy()

In [ ]:
df_submission_for_loader = sample_df.copy()
df_submission_for_loader["StudyInstanceUID"]=df_submission_for_loader["StudyInstanceUID"].apply(append_ext)

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=df_submission_for_loader,
directory='/kaggle/input/ranzcr-clip-catheter-line-classification/test/',
x_col="StudyInstanceUID",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(224,224))

In [ ]:
y_preds = model.predict(test_generator)

In [ ]:
df_submission.iloc[:, 1:] = y_preds
df_submission.head()

In [ ]:
df_submission.to_csv('submission.csv',index=False)